In [1]:
import re
import nltk
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
from nltk.corpus import stopwords
from collections import defaultdict
import spacy
import textacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [2]:
# Read in data
import pickle
with open('data/reviews_clean.pkl', 'rb') as f:
    reviews = pickle.load(f)
    f.close()

In [3]:
for r in reviews['review_text']:
    if 'lb' in r:
        print(r)
        break

I'm 5"5' and 125 lbs. i ordered the s petite to make sure the length wasn't too long. i typically wear an xs regular in retailer dresses. if you're less busty (34b cup or smaller), a s petite will fit you perfectly (snug, but not tight). i love that i could dress it up for a party, or down for work. i love that the tulle is longer then the fabric underneath.


In [4]:
onestar = list(reviews[reviews['star_rating']==5]['review_text'])
print(onestar[0])

Love this dress!  it's sooo pretty.  i happened to find it in a store, and i'm glad i did bc i never would have ordered it online bc it's petite.  i bought a petite and am 5'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.


### Rules for Text Cleaning:
- Short hand abbrev:
    + bc -> because
- Repeating words (sooo):
    + regex??

In [5]:
# add special case rule
from spacy.attrs import ORTH, LEMMA, POS

special_case = [{ORTH: u"bc", LEMMA: u"because", POS: u"CONJ"}]
nlp.tokenizer.add_special_case(u"bc", special_case)

suffixes = nlp.Defaults.suffixes + (r'''lb|cm|\"''',)
suffix_regex = spacy.util.compile_suffix_regex(suffixes)
nlp.tokenizer.suffix_search = suffix_regex.search

In [ ]:
text = """I'm 5"5' and 125 lbs. i ordered the s petite to make sure the length wasn't too long. i typically wear an xs regular in retailer dresses. if you're less busty (34b cup or smaller), a s petite will fit you perfectly (snug, but not tight). i love that i could dress it up for a party, or down for work. i love that the tulle is longer then the fabric underneath.
"""
textacy.preprocess.preprocess_text(text, False, 
                                              lowercase=True, no_urls=True, 
                                              no_emails=True, no_phone_numbers=True, 
                                              no_numbers=True, no_currency_symbols=True, 
                                              no_punct=True, no_contractions=False, 
                                              no_accents=True)

In [12]:
def clean_text(text):
    # remove unit of measurement
    text = re.sub('lb[s]?', '', text)
    text = re.sub('[0-9]{1,2}[\w]*', '', text)
    
    # abbreviations
    text = re.sub('imo', 'in my opinion', text)
    text = re.sub('bc', 'because', text)
    # slang
    text = re.sub('tad', 'litte', text)
    # size
    text = re.sub('x[x|s]', 'size', text)
    
    text = textacy.preprocess.preprocess_text(text, False, 
                                              lowercase=True, no_urls=True, 
                                              no_emails=True, no_phone_numbers=True, 
                                              no_numbers=True, no_currency_symbols=True, 
                                              no_punct=True, no_contractions=True, 
                                              no_accents=True)
    text = re.sub('number', '', text)
    return text

In [13]:
reviews['clean_text'] = reviews['review_text'].progress_apply(clean_text)

In [14]:
reviews.to_pickle('data/reviews_processed.pkl')

In [7]:
reviews = pd.read_pickle('data/reviews_processed.pkl')

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
analyzer = TfidfVectorizer().build_analyzer()
stemmer = nltk.PorterStemmer()
def stemmed_words(doc):
    return list(map(stemmer.stem, analyzer(doc)) )

pos_interest = ['ADJ','NOUN', 'VERB', 'ADV']
def trim_features(doc):
    review = nlp(doc)
    tokens = []
    for t in review:
        if not t.is_stop and t.is_alpha and t.pos_ in pos_interest:
            t_stem = stemmer.stem(t.text)
            if len(t_stem) > 2:
                tokens.append(t_stem)
    return tokens

In [ ]:
#12613 unique lemmas
#9694 unique Porter stems
#8190 unique Lancaster stems

Will go with Porter stemming because not much difference to Lancaster while retaining more meaning of words

Only considers top 3000 stems

In [16]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), 
                             stop_words='english',
                             analyzer=trim_features,
                             max_features=2000,
                             norm='l1')
corpus = list(reviews['clean_text'])

X = vectorizer.fit_transform(tqdm(corpus, desc='Calculating TF-iDF', total=len(reviews)))

In [ ]:
X.shape

In [17]:
with open('data/term_matrix.pkl', 'wb') as f:
    pickle.dump(X, f)
    f.close()

In [18]:
with open('data/vocab_key.pkl', 'wb') as f:
    pickle.dump(vectorizer.vocabulary_, f)
    f.close()